<a href="https://colab.research.google.com/github/mhtabkrklt/NLP/blob/main/mahamat_nlp_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(palette='summer')

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install evaluate

In [ ]:
import transformers
from datasets import load_dataset
import evaluate
from sklearn.model_selection import train_test_split

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Подготовка данных

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet', 'ca_test': 'data/ca_test-00000-of-00001.parquet'}
billsum = pd.read_parquet("hf://datasets/FiscalNote/billsum/" + splits["ca_test"])
billsum.head()

,text,summary,title
0,The people of the State of California do enact...,Existing property tax law establishes a vetera...,An act to amend Section 215.1 of the Revenue a...
1,The people of the State of California do enact...,Existing law provides that the Board of Parole...,"An act to amend Section 3550 of, and to add Se..."
2,The people of the State of California do enact...,The Sales and Use Tax Law imposes a tax on ret...,An act\nto add Chapter 3.8 (commencing with Se...
3,The people of the State of California do enact...,"Existing law requires all moneys, except for f...","An act to amend Sections 75220, 75221, and 752..."
4,The people of the State of California do enact...,Existing law defines a request regarding resus...,An act to add and repeal Section 4788 of the P...


In [ ]:
len(billsum)

1237

In [ ]:
from datasets import Dataset
billsum = Dataset.from_pandas(billsum)

In [ ]:
billsum = billsum.train_test_split(test_size=0.1)

In [ ]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 124
    })
})

In [ ]:
# Загрузка предобученного токенизатора для модели BART-bas
tokenizer = transformers.AutoTokenizer.from_pretrained("ainize/bart-base-cnn")

In [ ]:
print(type(billsum))

<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("ainize/bart-base-cnn")

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# Настройки обучения для Seq2Seq-модели
training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=2,
        report_to="none",
    )

In [ ]:
print(tokenized_billsum)  #какие ключи/колонки есть в датасете

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 124
    })
})


In [ ]:
# Инициализация тренера для Seq2Seq-модели
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-38-199256a528cd>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.018363
2,2.165600,1.967687


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=558, training_loss=2.151993905344317, metrics={'train_runtime': 424.0101, 'train_samples_per_second': 5.25, 'train_steps_per_second': 1.316, 'total_flos': 1357273356042240.0, 'train_loss': 2.151993905344317, 'epoch': 2.0})

# Предсказания на тестовом множестве

In [ ]:
text_example = billsum["test"]['text'][0]
print(text_example)

The people of the State of California do enact as follows:


SECTION 1.
Section 1253.3 of the Unemployment Insurance Code is amended to read:
1253.3.
(a) Notwithstanding any other provision of this division, unemployment compensation benefits, extended duration benefits, and federal-state extended benefits are payable on the basis of service to which Section 3309(a)(1) of the Internal Revenue Code applies, in the same amount, on the same terms, and subject to the same conditions as benefits payable on the basis of other service subject to this division, except as provided by this section.
(b) Benefits specified by subdivision (a) based on service performed in the employ of a nonprofit organization, or of a public entity, as defined by Section 605, with respect to service in an instructional, research, or principal administrative capacity for an educational institution are not payable to any individual with respect to any week which begins during the period between two successive academ

In [ ]:
#Токенизация и подготовка входных данных для модели
input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True,
    ).to(device)

NameError: name 'tokenizer' is not defined

In [ ]:
input_ids.shape

torch.Size([1, 1024])

In [ ]:
#Генерация текста моделью
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    max_length=142,
    min_length=56,
    num_beams=4,
)

In [ ]:
summary_text_ids

tensor([[    2,     0,  9089, 15528,   488,  1639,    14,  5755,  4660,  1795,
             6,  3112, 13428,  1795,     6,     8,   752,    12,  4897,  3112,
          1795,    32, 21467,    15,     5,  1453,     9,   544,     7,    61,
             5, 18387,  5833,  8302, 11459,     6,    11,     5,   276,  1280,
             6,    15,     5,   276,  1110,     6,     8,  2087,     7,     5,
           276,  1274,    25,  1795, 21467,    15,    10,  1453,     9,    97,
           544,  2087,     7,    42,  2757,     6,  4682,    25,  1286,     4,
          3015, 15528,   488,    67,  1639,    14,  1795, 17966,    30, 28764,
            36,   102,    43,   716,    15,   544,  3744,    11,     5, 12735,
             9,    10,  6651,  1651,     6,    50,     9,    10,   285, 10014,
             6,    25,  6533,     6,    19,  2098,     7,   544,    11,    41,
         35289,     6,   557,     6,    50,  5402,  6833,  2148,    13,    41,
          5984,  6786,    32,    45, 21467,     7,  

In [ ]:
#Декодирование выходных токенов в текст
decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

In [ ]:
len(decoded_text), len(text_example)

(706, 8060)

In [ ]:
decoded_text

'Existing law provides that unemployment compensation benefits, extended duration benefits, and federal-state extended benefits are payable on the basis of service to which the Internal Revenue Code applies, in the same amount, on the same terms, and subject to the same conditions as benefits payable on a basis of other service subject to this division, except as provided. Existing law also provides that benefits specified by subdivision (a) based on service performed in the employ of a nonprofit organization, or of a public entity, as defined, with respect to service in an instructional, research, or principal administrative capacity for an educational institution are not payable to any individual'

In [ ]:
#Пакетное создание суммаризаций для тестовых текстов через beam search с параметрами [max=142, min=56]
summaries = []

for text in tqdm(billsum["test"]['text']):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        ).to(device)

    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        max_length=142,
        min_length=56,
        num_beams=4,
    )

    decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    summaries.append(decoded_text)

  0%|          | 0/124 [00:00<?, ?it/s]

# Считаем качество(ROUGE)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=716480f395faff2446c163b7f5dccb557f76e147f5204124fecd83a145450e7d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
%%time
results = rouge.compute(
        predictions=summaries,
        references=billsum["test"]['summary']
    )

CPU times: user 4.52 s, sys: 14.9 ms, total: 4.54 s
Wall time: 10.3 s


In [ ]:
results

{'rouge1': np.float64(0.33356530091882763),
 'rouge2': np.float64(0.18769783062610373),
 'rougeL': np.float64(0.22388302064252552),
 'rougeLsum': np.float64(0.28580818644678463)}